In [167]:
import numpy as np
import random
import math

In [175]:
# Initial conditions
n = 10 # number of trash
m = 5 # size of grid
k = 3 # number of agents
mu = [3, 2] # center of trash pile
sig = [[0.1, 0], [0, 0.1]] # spread of pile
var = 0.1 # error in measurements

field = np.zeros((m,m))# array of trash pieces dispersed on field
visits = np.zeros((k,m,m))# array to track number of visits to each gridpoint
reward = np.zeros((k,m,m))# array of sum of rewards from each gridpoint
expected_mean = np.zeros((k,m,m)) # array of expected mean of each gridpoint

T = n # number of time steps
delta = 1 # amount reward decreases per visit
xi = 2 # constant xi > 1
gamma = 3 # max message length

messageSent = {}# dict {agent: list of messages to send - tuple [agent, time, arm, reward]}
messageRec = {}# dict {agent: list of messages received - tuple [agent, time, arm, reward]}


In [176]:
for i in range(k):
    messageSent[i] = []
    messageRec[i] = []

In [177]:
# Initialize communication network
# If graph[i][j] = 1 then there is an edge between agents i and j
# Array must be symmetric

# graph = np.ones((k,k))

graph = np.array([[1, 1, 0], [1, 1, 1], [0, 1, 1]])


In [178]:
# Initialize field
trash = np.random.multivariate_normal(mu, sig, n)
for i in range(n):
    x = int(trash[i][0])
    y = int(trash[i][1])
    field[x][y] += 1
    
print(field)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 2. 2. 0. 0.]
 [0. 3. 3. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [179]:
# Initialize agents
t = 0
for i in range(k):
    for x in range(m):
        for y in range(m):
            measure = round(np.random.normal(field[x][y], var),1)
            reward[i][x][y] += measure
            visits[i][x][y] += 1
    
    # Calculate expected mean
    expected_mean[i] = np.divide(reward[i], visits[i])
    

In [180]:
t += 1
while np.max(field) > 0:
    print("t: {}".format(t))
    index = []
    for i in range(k):        
        # intialize agents
        if t == 0:
            x = random.randint(0,m-1)
            y = random.randint(0,m-1)
            measure = np.round(np.random.normal(field[x][y], var), 1)
            ind = (x, y)            
            
        # Select arm based on maximized Q
        else:
            Q = expected_mean[i] + var*np.divide(math.sqrt(2*(xi + 1)*math.log(t)),np.sqrt(visits[i]))
            ind = np.unravel_index(np.argmax(Q), Q.shape)
            measure = np.round(np.random.normal(field[ind], var),1)
            index.append(ind)
            
        print(ind)
        print(Q)
        print(measure)
        
        # Make message to send to neighboring agents
        message = (t, i, ind, measure)
        send = messageSent[i]
        if len(send) == gamma:
            send.pop(0)
        send.append(message)
        messageSent[i] = send
        
    # Pick up trash from selected grid points
    for i in range(len(index)):
        ind = index[i]
        field[ind] -= 1
                
    
    # receive messages from neighbors and adjust expected mean
    for i in range(k):
        print(i)
        new_visits= np.zeros((m,m))
        # compare new messages with history of messages and skip any repeats
        for j in range(k):
            if graph[i][j] == 1:
                received = messageRec[i]
                sent = messageSent[j]
                for l in range(len(sent)):
                    msg = sent[l]
                    if received.count(msg) == 0:
                        received.append(msg)
                        ind = msg[2]
                        visits[i][ind] += 1
                        new_visits[ind] += 1
                        reward[i][ind] += msg[3]
                messageRec[i] = received
        
        # calculate expected mean
        reward[i] -= np.multiply(new_visits, visits[i])
        # expected_mean[i] = (expected_mean[i] -1) + np.divide(new_reward[i], visits[i])
        expected_mean[i] = np.divide(reward[i], visits[i])
        print(reward[i])
        print(visits[i])
        print(expected_mean[i])
    
    print(field)
    t += 1 



    

t: 1
(3, 1)
[[-0.1 -0.1  0.  -0.1 -0.1]
 [ 0.1  0.1 -0.1  0.1  0.1]
 [-0.1  2.2  2.   0.  -0.1]
 [ 0.   3.   2.9  0.   0. ]
 [-0.1 -0.1  0.1 -0.1 -0.1]]
2.9
(3, 1)
[[ 0.   0.  -0.1  0.   0. ]
 [ 0.   0.   0.  -0.1  0. ]
 [ 0.   1.9  1.9  0.1  0.2]
 [-0.1  3.   2.9  0.1  0. ]
 [ 0.  -0.1  0.1  0.   0.1]]
2.9
(3, 2)
[[ 0.1  0.2  0.1  0.   0.1]
 [ 0.  -0.1 -0.1  0.2  0.2]
 [-0.2  2.   2.   0.3  0.1]
 [ 0.1  3.   3.1  0.  -0.1]
 [ 0.  -0.1 -0.2 -0.2  0. ]]
3.0
0
[[-0.1 -0.1  0.  -0.1 -0.1]
 [ 0.1  0.1 -0.1  0.1  0.1]
 [-0.1  2.2  2.   0.  -0.1]
 [ 0.   2.8  2.9  0.   0. ]
 [-0.1 -0.1  0.1 -0.1 -0.1]]
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 3. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
[[-0.1        -0.1         0.         -0.1        -0.1       ]
 [ 0.1         0.1        -0.1         0.1         0.1       ]
 [-0.1         2.2         2.          0.         -0.1       ]
 [ 0.          0.93333333  2.9         0.          0.        ]
 [-0.1        -0.1         0.1        -0.1        -0.

In [181]:
print(field)
print(t)

[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0. -1. -1.  0.  0.]
 [ 0. -1. -2.  0.  0.]
 [ 0.  0.  0.  0.  0.]]
6
